In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from collections import defaultdict

In [8]:
complete_data = pd.read_csv("datathon_2024_dataset.csv")

In [9]:
complete_data = complete_data[['game_date','home_team','away_team','home_score','away_score']]

In [10]:
final_data = []

In [11]:
complete_data['game_date'] = complete_data['game_date'].astype(str)
for i in range(2000,2024):
    year = str(i)
    filtered_data = complete_data[complete_data['game_date'].str[:4] == year]
    filtered_data['game_date'] = pd.to_datetime(filtered_data['game_date'], format='%Y%m%d')
    
    scoring_hashmap = defaultdict(int)
    conceding_hashmap = defaultdict(int)

    for index, row in filtered_data.iterrows():
        filtered_data.at[index, 'home_team_total_runs_scored'] = scoring_hashmap[row['home_team']]
        scoring_hashmap[row['home_team']] += row['home_score']
        filtered_data.at[index, 'away_team_total_runs_scored'] = scoring_hashmap[row['away_team']]
        scoring_hashmap[row['away_team']] += row['away_score']
        filtered_data.at[index, 'home_team_total_runs_conceded'] = conceding_hashmap[row['home_team']]
        conceding_hashmap[row['home_team']] += row['away_score']
        filtered_data.at[index, 'away_team_total_runs_conceded'] = conceding_hashmap[row['away_team']]
        conceding_hashmap[row['away_team']] += row['home_score']
        
    post_scoring_hashmap = defaultdict(int)
    post_conceding_hashmap = defaultdict(int)

    for index, row in filtered_data.iterrows():
        post_scoring_hashmap[row['home_team']] += row['home_score']
        filtered_data.at[index, 'home_team_total_runs_scored_post'] = post_scoring_hashmap[row['home_team']]
        post_scoring_hashmap[row['away_team']] += row['away_score']
        filtered_data.at[index, 'away_team_total_runs_scored_post'] = post_scoring_hashmap[row['away_team']]
        post_conceding_hashmap[row['home_team']] += row['away_score']
        filtered_data.at[index, 'home_team_total_runs_conceded_post'] = post_conceding_hashmap[row['home_team']]
        post_conceding_hashmap[row['away_team']] += row['home_score']
        filtered_data.at[index, 'away_team_total_runs_conceded_post'] = post_conceding_hashmap[row['away_team']]
        
    final_data.append(filtered_data)
final_df = pd.concat(final_data, ignore_index=True)

C:\Users\kusha\AppData\Local\Temp\ipykernel_12652\822856438.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['game_date'] = pd.to_datetime(filtered_data['game_date'], format='%Y%m%d')
C:\Users\kusha\anaconda3\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\kusha\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [12]:
filtered_data = final_df

In [13]:
for index, row in filtered_data.iterrows():
    hrs = row['home_team_total_runs_scored']
    hrc = row['home_team_total_runs_conceded']
    ars = row['away_team_total_runs_scored']
    arc = row['away_team_total_runs_conceded']
    
    if (hrs**2 + hrc**2) == 0:
        filtered_data.at[index, 'home_EWP'] = 0
    else:
        filtered_data.at[index, 'home_EWP'] = hrs**2/(hrs**2 + hrc**2)
    if (ars**2 + arc**2) == 0:
        filtered_data.at[index, 'away_EWP'] = 0
    else:
        filtered_data.at[index, 'away_EWP'] = ars**2/(ars**2 + arc**2)

In [14]:
for index, row in filtered_data.iterrows():
    hrs = row['home_team_total_runs_scored']
    hrc = row['home_team_total_runs_conceded']
    ars = row['away_team_total_runs_scored']
    arc = row['away_team_total_runs_conceded']
    
    if row['away_EWP'] == 0:
        filtered_data.at[index, 'rel_home_EWP'] = 0
    else:
        filtered_data.at[index, 'rel_home_EWP'] = row['home_EWP']/row['away_EWP']
    if row['home_EWP'] == 0:
        filtered_data.at[index, 'rel_away_EWP'] = 0
    else:
        filtered_data.at[index, 'rel_away_EWP'] = row['away_EWP']/row['home_EWP']

In [15]:
for index, row in filtered_data.iterrows():
    hrsp = row['home_team_total_runs_scored_post']
    hrcp = row['home_team_total_runs_conceded_post']
    arsp = row['away_team_total_runs_scored_post']
    arcp = row['away_team_total_runs_conceded_post']
    
    if (hrsp**2 + hrcp**2) == 0:
        filtered_data.at[index, 'home_EWP_post'] = 0
    else:
        filtered_data.at[index, 'home_EWP_post'] = hrsp**2/(hrsp**2 + hrcp**2)
    if (arsp**2 + arcp**2) == 0:
        filtered_data.at[index, 'away_EWP_post'] = 0
    else:
        filtered_data.at[index, 'away_EWP_post'] = arsp**2/(arsp**2 + arcp**2)

In [16]:
for index, row in filtered_data.iterrows():
    hrsp = row['home_team_total_runs_scored_post']
    hrcp = row['home_team_total_runs_conceded_post']
    arsp = row['away_team_total_runs_scored_post']
    arcp = row['away_team_total_runs_conceded_post']
    if row['away_EWP_post'] == 0:
        filtered_data.at[index, 'rel_home_EWP_post'] = 0
    else:
        filtered_data.at[index, 'rel_home_EWP_post'] = row['home_EWP_post']/row['away_EWP_post']
    if row['home_EWP_post'] == 0:
        filtered_data.at[index, 'rel_away_EWP_post'] = 0
    else:
        filtered_data.at[index, 'rel_away_EWP_post'] = row['away_EWP_post']/row['home_EWP_post']

In [17]:
for index, row in filtered_data.iterrows():
    rhewp = row['rel_home_EWP']
    rhewpp = row['rel_home_EWP_post']
    raewp = row['rel_away_EWP']
    raewpp = row['rel_away_EWP_post']
    
    filtered_data.at[index,'diff_rel_home_EWP'] = rhewpp - rhewp
    filtered_data.at[index,'diff_rel_away_EWP'] = raewpp - raewp

In [18]:
print(filtered_data)

       game_date home_team away_team  home_score  away_score  \
0     2000-03-29       NYN       CHN           3           5   
1     2000-03-30       CHN       NYN           1           5   
2     2000-04-03       ATL       COL           2           0   
3     2000-04-03       CIN       MIL           3           3   
4     2000-04-03       FLO       SFN           6           4   
...          ...       ...       ...         ...         ...   
56770 2023-10-01       CHA       SDN           1           2   
56771 2023-10-01       DET       CLE           5           2   
56772 2023-10-01       KCA       NYA           5           2   
56773 2023-10-01       SEA       TEX           1           0   
56774 2023-10-01       TOR       TBA           8          12   

       home_team_total_runs_scored  away_team_total_runs_scored  \
0                              0.0                          0.0   
1                              5.0                          3.0   
2                             

In [19]:
filtered_data.to_csv('performance_metric_final.csv')